In [1]:
from torch import float32, max, no_grad, save, device
from torch.backends import mps
from torch.nn import (
    Conv2d,
    CrossEntropyLoss,
    Flatten,
    Linear,
    MaxPool2d,
    Module,
    ReLU,
    Sequential,
    Softmax,
)
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.transforms.v2 import Compose, Normalize, ToDtype, ToImage

In [2]:
# Define the transformation to apply to the data
transform = Compose(
    [
        ToImage(),
        ToDtype(float32, scale=True),
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# Load the CIFAR10 dataset
train_data = CIFAR10("cifar10_data", download=True, train=True, transform=transform)
test_data = CIFAR10("cifar10_data", download=True, train=False, transform=transform)

# Split the training set into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = random_split(train_data, [train_size, val_size])

# Define the data loaders for the training, validation, and testing sets
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [01:06<00:00, 2564419.91it/s]


Extracting cifar10_data/cifar-10-python.tar.gz to cifar10_data
Files already downloaded and verified


In [3]:
class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layers = Sequential(
            Conv2d(3, 32, kernel_size=3, padding="same", stride=1),  # 32, 32, 32
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),  # 16, 16, 32
            Conv2d(32, 64, kernel_size=3, padding="same", stride=1),  # 16, 16, 64
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),  # 8, 8, 64
            Flatten(),
            Linear(64 * 8 * 8, 128),
            ReLU(inplace=True),
            Linear(128, 10),
            Softmax(dim=1),
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [4]:
device = device("mps" if mps.is_available() else "cpu")
model = CNN().to(device)

In [5]:
device

device(type='mps')

In [6]:
# Define hyperparameters
epochs = 10

# Initialize optimizer
optimizer = Adam(model.parameters())

# Define loss function
criterion = CrossEntropyLoss()

In [7]:
# Train the model
model.train()
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

save(model.state_dict(), "../../../Models/cifar10_cnn.pth")

Epoch [1/10], Loss: 1.9912
Epoch [2/10], Loss: 1.9557
Epoch [3/10], Loss: 1.8989
Epoch [4/10], Loss: 1.9004
Epoch [5/10], Loss: 2.0212
Epoch [6/10], Loss: 1.9408
Epoch [7/10], Loss: 1.7197
Epoch [8/10], Loss: 1.7749
Epoch [9/10], Loss: 1.8561
Epoch [10/10], Loss: 1.6631


In [8]:
model.eval()

# Track validation accuracy
correct = 0
total = 0

with no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {(correct / total):.4f}")

In [ ]:
model.eval()

# Track test accuracy
correct = 0
total = 0

with no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(correct / total):.4f}")